# Introduction to time series analysis

This notebook provides an overview of time series analysis. Time series are a very common type of data. Applications of time series analysis include:

- **Demand forecasting:** electricity production, internet bandwidth, traffic management, inventory management
- **Medicine:** time dependent treatment effects, IoT monitoring devices
- **Engineering and science:** signal analysis, analysis of physical processes
- **Capital markets and economics:** seasonal unemployment, risk analysis, financial analysis

In this lesson you will learn the following:

- Basic properties of time series
- How to perform and understand decomposition of time series
- Modeling of time series residuals and the ARIMA model
- Forecasting time series values from models

What sets **time series data** apart from the kind of data we have been dealing with so far (sometimes referred to as **cross-sectional data**) is that in time series data, we are really only interested in predictions *beyond the range* of available data. This means that we look at the past not so much to model the past, but rather to predict the future, which we also call **forecasting**. We say that forecasting is a type of **extrapolation**, whereas in predicting in general is usually a type of **interpolation**. Of course we may also need to extrapolate with cross-sectional data, but with time series data extrapolation is the rule, not the exception. As we will see, in practice, many of the methods we learned for linear regression can still be employed here, but with some modification.

Another important new word we encounter in this notebook in the the word **process**: In the past notebooks, generating random samples from a distribution was called **Monte Carlo simulations**. In the context of time series data where each sample generated may depend on the previous ones, we use the word **process** instead, and we say the data was generated using a **stochastic process** where the word stochastic means that it's probability-based and hence noisy (the opposite of the word **deterministic**). So we say that we are generating a sequence of random numbers based on a stochastic process, i.e. a probability rule that links each instance to the previous ones.

## Working with time series in `pandas`

There are significant capabilities in `pandas` for working with time series data. The trick is to use the timestamp as row index for the data. Let's start with a simple univariate time series.

In [1]:
from math import sin, pi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as nr

import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error

import statsmodels.graphics.tsaplots as splt
import statsmodels.api as statsmodels
import statsmodels.formula.api as sm
import statsmodels.tsa.seasonal as sts
import statsmodels.tsa.arima_process as arima_process
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 5]

import warnings
warnings.filterwarnings('ignore')

Here are two helper functions we will be referring to several time during the notebook.

In [2]:
def plot_acf_pacf(x, lags = 40):
    x = x[x.notna()] # remove NAs
    fig, axes = plt.subplots(1, 2, figsize = (15, 5))
    fig = splt.plot_acf(x, lags = lags, ax = axes[0])
    fig = splt.plot_pacf(x, lags = lags, ax = axes[1]);
    return None

def plot_ts_resid(x):
    x = x[x.notna()] # remove NAs
    fig, axes = plt.subplots(1, 2, figsize = (15, 5))
    fig = sns.lineplot(x.index, x, ax = axes[0])
    fig = sns.distplot(x, ax = axes[1]);
    return None

Let's now start with some series of numbers.

In [3]:
ts = pd.Series([sin(x/30.0) for x in range(366)])
ts.head(5)

0    0.000000
1    0.033327
2    0.066617
3    0.099833
4    0.132939
dtype: float64

The object we created is a series of floating point values. However, this is not yet a time series. For that, we need to add an index.

In [4]:
ts.index = pd.date_range(start = '1-1-2016', end = '12-31-2016', freq = 'D')
ts.head(5)

2016-01-01    0.000000
2016-01-02    0.033327
2016-01-03    0.066617
2016-01-04    0.099833
2016-01-05    0.132939
Freq: D, dtype: float64

You can see that the index is now the date-time for each value. Let's plot the time series.

In [5]:
sns.lineplot(x = ts.index, y = ts);

You can see that the time axis is labeled automatically, and the tick marks on the x-axis are made sparse enough so as not to overcrowd the plot.

Since the index is a timestamp, we can slice the data using timestamp cut-offs:

In [6]:
ts_slice = ts['1/1/2016':'6/30/2016']
sns.lineplot(x = ts_slice.index, y = ts_slice);

Notice that the plot covers the subset of dates specified. 

## White noise

In this section we will explore some basic properties time series. Understanding these properties will help in understanding the models we explore in subsequent sections.

A series $\{X_t\} = \{X_1, X_2, \cdots\}$ of **independent identically distributed (IID)** random variables $X_t \sim N(0, \sigma)$ drawn from a normal distribution is said to be a **white noise** series. Since the series is IID there is no correlation from one value to the next. Usually $t$ is descrete and evenly-spaced, such as $t = 1, 2, \dots$, but $t$ can also be discrete but unevenly spaced, or even continuous. In practice, when the time variable is continuous, we end up sampling it at discrete and usually evenly-spaced time points so it becomes discrete.

If we successfully model the time series so that any *trend* in the data is explained away, then what remains should look like white noise. To **detrend** time series data, we need to capture all time-dependent relationships that exist in the data.

In [7]:
index = pd.date_range(start = '1-1-2015', end = '12-31-2015', freq = 'D')
wn = pd.Series(nr.normal(size = len(index)), index = index)
plot_ts_resid(wn);

Notice that the values of the time series wander randomly around zero, with no particular trend. As expected, the values of the white noise series are normally distributed.

The values of the white noise series are IID, so we do not expect the values to show any dependency over time. In time series analysis we measure dependency using **auto-correlation**. Auto-correlation is the correlation of a series with itself **lagged** (offset in time) by some number of time steps. Auto-correlation at lag $k$, denoted by $\rho_k$ is the correlation between $X_t$ and $X_{t-k}$ for all times $t$. Notice that for any series, $\rho_0 = 1$.

We can also define a second order **partial auto-correlation**. The partial autocorrelation at lag $k$ is the correlation that remains after we remove the effect of any correlations due to the terms $X_{t-i}$ for $i<k$. We can think of this as being the **direct effect** at lag $k$ after removing all indirect effects at lags $i = 1, \dots, k-1$. There are more computationally efficient ways to calculate partial auto-correlation, but the most intuitive way to do so is the following: If $X_t = \phi_1X_{t-1} + \phi_2X_{t-2}$ then $\phi_2$ is the partial auto-correlation at lag 2.

### Exercise

Let's read in the white noise series from above into a `DataFrame` and explore some time series functionality in `pandas`:

In [8]:
data_ts = pd.DataFrame({'x': wn})
data_ts.head()

,x
2015-01-01,-0.572644
2015-01-02,-0.727448
2015-01-03,2.265471
2015-01-04,-0.220980
2015-01-05,-0.232665


- Create two new features `x_roll` and `x_ewma` which are moving averages and exponentially weighted moving averages of `x`. Then plot a line plot of all three. HINT: Use the `rolling` and `ewm` methods. Explore their arguments to see how we can make the data more smooth or less smooth.

In [9]:
data_ts['x_roll'] = data_ts['x'].rolling(5).mean()
data_ts['x_ewma'] = data_ts['x'].ewm(1).mean()

- Create a new column called `x_lag_1` which is the lag of `x` (each row of `x_lag_1` is stores the value of `x` in the previous row. HINT: Use the `shift` method.

In [10]:
data_ts['x_lag_1'] = data_ts['x'].shift(1)

- Find the auto-correlations at lag $k = 2$ for the white-noise series in the above data.

In [11]:
data_ts['x_lag_2'] = data_ts['x'].shift(2)
data_ts.corr()

,x,x_roll,x_ewma,x_lag_1,x_lag_2
x,1.000000,0.429431,0.867034,-0.049251,-0.009366
x_roll,0.429431,1.000000,0.735011,0.428407,0.415823
x_ewma,0.867034,0.735011,1.000000,0.388512,0.183971
x_lag_1,-0.049251,0.428407,0.388512,1.000000,-0.051786
x_lag_2,-0.009366,0.415823,0.183971,-0.051786,1.000000


- Find the partial auto-correlation at lag $k = 2$ for the white-noise series. 

  HINT: To do this, you need to train a linear regression model without the intercept term. You can drop the intercept term from a model by adding `-1` to the formula argument in `sm.ols`.

In [12]:
model = sm.ols(formula = 'x ~ x_lag_1 + x_lag_2 -1', data = data_ts).fit()
print(model.summary2())

                        Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): -0.003   
Dependent Variable:     x                AIC:                         1030.4607
Date:                   2021-08-31 20:10 BIC:                         1038.2495
No. Observations:       363              Log-Likelihood:              -513.23  
Df Model:               2                F-statistic:                 0.4075   
Df Residuals:           361              Prob (F-statistic):          0.666    
R-squared (uncentered): 0.002            Scale:                       0.99537  
------------------------------------------------------------------------------------
              Coef.       Std.Err.         t         P>|t|        [0.025      0.975]
------------------------------------------------------------------------------------
x_lag_1      -0.0471        0.0526      -0.8952      0.3713      -0.1506      0.0564
x_lag_2      -0.0084        0.0528      -0.1

So you now know how to plot auto-correlation and partial auto-correlation at a given lag $k$. Let's plot the **auto-correlation function (ACF)** and **partial auto-correlation function (PACF)** of the data. The ACF and PACF are just the auto-correlations and partial auto-correlations at **all lags** $k = 1, 2, \dots, n$ for some maximum lag $n$. The ACF and PACF are plotted against the lag $k$ and a 95% confidence band around them is used to show which values are significant.

In [13]:
plot_acf_pacf(wn, lags = 40)

As expected the white noise series only has a significant auto-correlation and partial auto-correlation values at lag zero.

White noise is an example of a **stationary** time series. A stationary time series has constant mean, constant variance, and constant auto-correlation structure over time, but for stationary time series in general, the auto-correlations can be significant at lags $k > 0$.

**NOTE:** The `statsmodels` packages uses the engineering convention for displaying partial autocorrelation. The value at 0 lag, which always must be 1, is displayed. In many statistical packages, including R, this 0 lag value is not displayed. This difference in conventions can lead to a lot of confusion.

Let's see what happens when we add a trend and seasonality to a white noise series.

In [14]:
dates = pd.date_range(start = '1-1-2015', end = '12-31-2015', freq = 'D')
wn = pd.Series(nr.normal(scale = 5, size = len(dates)), index = dates)
plot_ts_resid(wn)

- Create a new series `wnt` composed of a trend $X_t = 0.5 \cdot t$ and the white noise from above. Then plot the series. HINT: To do this, first create an array called `trend` that is just an integer index of the timestamp `dates`. This array stores the values of $t$ in the above equation. You can then add the trend and white noise to get the new series.

In [15]:
t = np.array(range(len(dates)))
wnt = 0.5 * t + wn

- Plot the distribution of the new series. What can you conclude?

In [16]:
plot_ts_resid(wnt)

- Plot the ACF and PACF using the `plot_acf_pacf` function. Use up to 50 lags. Based on what you see, what can you conclude about the trend of the data?

In [17]:
plot_acf_pacf(wnt, lags = 40)

- Is a time series with a trend stationary?

- Let's now add periodicity to the white noise using the **sinusoidal component** `25 * np.sin(pi*x/6)`. Name the new series `wnp` and plot it.

In [18]:
wnp = 25 * np.sin(pi*t/6) + wn

- Plot the distribution of the new series. What can you conclude?

In [19]:
plot_ts_resid(wnp)

- To see what's happening next, it may help to zoom in on a narrower window of the above plot. Show the time series plot for the first 30 data points.

In [20]:
plot_acf_pacf(wnp, lags = 40)

- Plot the ACF and PACF of the time series plot up to 50 lags. Based on what you see, what can you conclude about the periodicity of the data?

In [21]:
plot_acf_pacf(wnp, lags = 40)

- Does the periodic behavior of the ACF and PACF continue or die out?

- Create a new series by adding the trend component, the seasonal component and the white noise together. Call this series `wntp`. Plot the series.

In [22]:
wntp = 25 * np.sin(pi*t/6) + wnt

- Plot the ACF and PACF of the series and compare it to the previous two.

In [23]:
plot_acf_pacf(wntp, lags = 60)

- Of the ACF and the PACF, which do you find more helpful in teaching us something about the data?

### End of exercise

## Random Walks

A **random walk** is the defined by the sum of a white noise series. In other words, if $\{w_t\}$ is a white noise series and $y_t = y_{t-1} + w_t$, then $\{y_t\}$ is a random walk. If we rewrite the series as $w_t = y_t - y_{t-1}$, we notice that we can get white noise by taking the **difference** $y_t - y_{t-1}$ of a random walk at lag 1.

A random walk has constant mean, but its the variance increases with time and it is un-bounded. Therefore, a random walk is **non-stationary**. You can see the erratic nature of a random walk by rerunning the cell below multiple times. Notice how the random walk is created by creating a white noise process and using the cumulative sum function `cumsum` on it.

In [24]:
dates = pd.date_range(start = '1-1-2015', end = '12-31-2015', freq = 'D')
wn = pd.Series(nr.normal(scale = 5, size = len(dates)), index = dates)
rw = wn.cumsum()

plot_ts_resid(rw)

Let's re-create the random walk a few different times and see how each random walk differs from the rest.

In [25]:
dates = pd.date_range(start = '1-1-2015', end = '12-31-2015', freq = 'D')

for _ in range(20):
    wn = pd.Series(nr.normal(scale = 5, size = len(dates)), index = dates)
    rw = wn.cumsum()
    sns.lineplot(rw.index, rw, alpha = 0.4);

A random walk is an example of a **Markov chain**, which is a stochastic process (a series of time-dependent random variables) where each value only depends on the previous one.

Let's also plot the ACF and PACF of the random walk for up to 40 lags.

In [47]:
plot_acf_pacf(rw)

The code in the cell below uses its `diff` method to perform first order differencing on the time series.

In [48]:
rw_diff = rw.diff()
plot_ts_resid(rw_diff)

So differencing the random walk appears to make it behave like white noise. We can also look at the ACF and PACF for further confirmation.

In [49]:
plot_acf_pacf(rw_diff)

Unsurprisingly we see the white noise process.

## STL decomposition of time series

We just learned one way that we can work with time series data: instead of trying to model the data itself, we just focus on modeling the differences. But what if the time series doesn't behave like a random walk. What if instead, we have a predictable trend and seasonality around it?

A direct decomposition model is known as **STL** which stands for **seasonal and trend decomposition using loess**. Not too surprisingly this model does the following:

- The trend is removed using a LOESS regression model, which is a non-linear regression method that builds on the ideas developed in linear regression.
- The seasonal component is removed using a regression on periodic components.
- The remainder is known as the residual and should behave like white noise. In times series analysis, the residuals are sometimes called **innovations**. Can you guess why?

The decomposition can be either **additive** or **multiplicative**. The additive model simply sums the components and is written:

$$TS(t) = S(t) + T(t) + R(t)$$

The multiplicative model multiplies the three components. This model is particularly useful in the common case where the seasonal effect increases in proportional to the trend. We can write this model as an additive model by applying a log transformation:

$$TS(t) = S(t) \cdot T(t) \cdot R(t)$$

Let's try this out on a time series which has a seasonal, trend, and white noise residual component. Here's the time series we will use it on:

In [ ]:
dates = pd.date_range(start = '1-1-2015', end = '12-31-2015', freq = 'D')
wn = pd.Series(nr.normal(scale = 5, size = len(dates)), index = dates) # white noise

t = np.array(range(len(dates)))
wnt = 0.5 * t + wn # white noise with trend
wntp = 25 * np.sin(pi*t/6) + wnt # white noise with trend and add seasonality

plot_ts_resid(wntp)

And here's what happens when we decompose it:

In [ ]:
def decomp_ts(ts, period, model = 'additive'):
    res = sts.seasonal_decompose(ts, model = model, period = period)
    return(pd.DataFrame({'ts': ts, 'trend': res.trend, 'seasonal': res.seasonal, 'resid': res.resid}, 
                        index = ts.index))

decomp = decomp_ts(wnp, period = 12)
decomp.head(10)

Note that there are missing values in the trend and residuals at the beginning but also at the end of the data. So to avoid running into problems, we will replace the missing values by back-filling the beginning and forward-filling the ending.

In [ ]:
decomp = decomp.bfill().ffill()

If we look at the ACF and PACF of the original series we can clearly see the seasonality. The trend is not as obvious, but we can see the trend by just plotting the time series.

In [ ]:
plot_acf_pacf(decomp['ts'], lags = 40)

Once we decompose the data, we can look at ACF and PACF of the residuals. This is what's left after removing trend and seasonality. If the decomposition worked well, then we should see residuals and look and behave like white noise.

In [ ]:
plot_acf_pacf(decomp['resid'], lags = 40)

Does it appear that the STL process has removed the trend and seasonal components of the time series fully or partially? With real data, it's hard to answer that, since some of the significant correlation values are significant by chance.

## ARIMA models for the residual series

Now that we have investigated the basic properties of time series and looked at how to decompose it, we can try to decompose any time series data and then focus our attention on modeling the residuals.

The class of models we will investigate are known and **autoregressive integrated moving average** or **ARIMA** model. We will work our way through each component of this model in the next few subsections. 

The ARIMA model and its relatives are **linear** in their coefficients. ARIMA models are in effect linear regression models. However, as you will see, ARIMA models are constructed to account for the serial correlations in time series data.

### Autoregressive model $\text{AR}(p)$

The values of an **autoregressive** or **AR** time series are determined by a linear combination of the past values. In other words, the AR model accounts for serial correlation in the values of the time series. We can write the value of an autoregressive series of **order** $p$ or $\text{AR}(p)$ series at time t as:

$$X_t = \alpha_1 X_{t-1} + \alpha_2 X_{t-2} + \dots + \alpha_p X_{t-p} + w_t$$

where $w_t$ is a white noise process.

An AR process has the following properties:

- The value of $\rho_0$ is always equal to 1.
- The auto-correlations can be written as $p_k = \alpha^k$.
- The number of nonzero PACF values is equal to $p$.

AR models are specifically for **stationary time series** (hence the importance of decomposing the series first). If the variance is not constant or the trend component has not been removed, AR models will not produce satisfactory results.

### Moving average model $\text{MA}(q)$

For a **moving average** or **MA** model the value of the time series at time `t` is determined by a linear combination of past white noise terms. In other words, the MA model accounts for auto-correlation in the noise terms. We can write the $\text{MA}(q)$ model as the linear combination of the last $q$ white noise terms $w_i$:

$$X_t = w_t + \beta_1 w_{t-1} + \beta_2 w_{t-2} + \cdots + \beta_q w_{t-q}$$

An MA process has the following properties:

- The value of $\rho_0$ is always 1.
- The number of nonzero $\rho_{k \ne 0}$ values is equal to $q$.

MA models are specifically for **stationary time series**. If the variance is not constant or the trend component has not been removed, MA models will not produce satisfactory results.

NOTE: It can be shown theoretically that an $\text{AR}(1)$ process is equivalent to an $\text{MA}(\infty)$ process under the assumption that $-1 < \alpha_1 < 1$. In fact, any stationary process can be represented as $\text{MA}(\infty)$. Similarly, an $\text{MA}(1)$ process is equivalent to an $\text{AR}(\infty)$ process.

### The Autoregressive moving average model $\text{ARMA}(p, q)$

We can combine the AR and MA models to create an **autoregressive moving average** or **ARMA** model. This model accounts for serial correlation in both noise terms and values. We would expect we can write an ARMA model of order $(p, q)$ as:

$$
X_t = \alpha_1 X_{t-1} + \alpha_2 X_{t-2} + \cdots + \alpha_p X_{t-p} + w_t + \beta_1 w_{t-1} + \beta_2 w_{t-2} + \cdots + \beta_q w_{t-q}
$$

Intuitively, we can think of the AR component of an ARMA model as capturing the previous trends and the MA components as capturing "shocks".

### Autoregressive integrated moving average model and beyond

The **autoregressive integrated moving average (ARIMA)** model adds an integrating term to the ARMA model. The integrating component performs differencing to model a random walk component. The integrating component models one of the **non-stationary** parts of a time series. The ARIMA model is defined by orders $(p, d, q)$. We have already looked at $\text{AR}(p)$ and $\text{MA}(q)$ models. The order of the differencing operator of the integrating term is defined by $d$. Since the integrating term is a differencing operator, there is no coefficient to estimate.

We can further extend the ARIMA model by letting it model seasonality-related components. The result is called a Seasonal ARIMA or **SARIMA** model, written as $\text{SARIMA}(p, d, q)(P, D, Q)_m$. The parameters $(p, d, q)$ are from ARIMA, and the parameters $(P, D, Q)$ their equivalent counterparts, but apply to the seasonal component. Finally, $m$ is the length of the peroid that makes the data seasonal.

### ARMA example

Let's now look at some examples. We will only illustrate the the $\text{ARMA}(p, q)$ model here, and leave it to the reader to try $\text{ARIMA}(p, d, q)$ and $\text{SARIMA}(p, d, q)(P, D, Q)_m$ models, since the code looks very similar at a high-level, but has more arguments to account for all the parameters.

In the following section, we first generate an ARMA **process**. This means we fix $\alpha_i$ and $\beta_i$ in the equation below, then randomly generate the first element of the sequence, feed it through the equation and end up with a sequence of random numbers.

$$
X_t = \alpha_1 X_{t-1} + \alpha_2 X_{t-2} + \cdots + \alpha_p X_{t-p} + w_t + \beta_1 w_{t-1} + \beta_2 w_{t-2} + \cdots + \beta_q w_{t-q}
$$

To make this easier, we write a function called `ARMA_process` to do this. Further down you will see another function called `ARMA_model`: this function will take the data generated by `ARMA_process` and tries to model it using an $\text{ARMA}(p, q)$ model with $p$ and $q$ specified by us. We can then see how close the model's estimates of the coefficients (let's call them $\hat \alpha_i$ and $\hat \beta_i$) come to the actual coefficents used in the process $\alpha_i$ and $\beta_i$.

In [ ]:
nr.seed(4477)
def ARMA_process(ar_coef = None, ma_coef = None, start = '1-2005', end = '1-2015'):
    dates = pd.date_range(start = start, end = end, freq = 'M')
    # we need a leading 1 for AR and MA coefficients
    if ar_coef is None:
        ar_coef = [1]
    else:
        ar_coef = [1] + [-i for i in ar_coef] # AR coefficients need to be multiplied by -1
    if ma_coef is None:
        ma_coef = [1]
    else:
        ma_coef = [1] + ma_coef
    ts = arima_process.ArmaProcess(ar_coef, ma_coef)
    print('Is the time series stationary? ' + str(ts.isstationary))
    print('Is the time series invertible? ' + str(ts.isinvertible))
    return(pd.Series(ts.generate_sample(120), index = dates))

So here's how we can generate an $\text{AR}(2)$ model with $\alpha_1 = 0.75$ and  $\alpha_2 = 0.25$:

In [35]:
ts_series_ar2 = ARMA_process(ar_coef = [.75, .25])

Is the time series stationary? False
Is the time series invertible? True


Notice that we have applied some tests to determine if our choice of $AR(2)$ coefficients are stable. 

In [36]:
plot_ts_resid(ts_series_ar2)

The question now is, what are the statistical properties of this $\text{AR}(2)$ process? Let's look at some plots.

In [37]:
plot_acf_pacf(ts_series_ar2, lags = 40)

The $\text{AR}(2)$ process produces a series with significant correlations in the lags, as shown in the  ACF plot. More importantly, the PACF has 2 significant non-zero lag values, consistent with an $\text{AR}(2)$ model.

The AR time series model estimates the coefficients for the order of the model specified. The code in the cell below does the following:
1. Uses the `ARIMA` function from the `statsmodels` package to define model. The order of  the AR model is specified as `(p, 0, 0)`.
2. Fits the coefficient values using the `fit` method on the model object.
3. Prints the output of the `summary` method, showing useful statistics to understand the model.  

Run this code and examine  the  properties of the  results. 

In [38]:
def model_ARIMA(ts, order):
    from statsmodels.tsa.arima_model import ARIMA
    try: 
        model = ARIMA(ts, order = order)
        model_fit = model.fit(disp = 0, method = 'mle', trend = 'nc', start_ar_lags = 7)
        print(model_fit.summary2())
        return model_fit
    except ValueError:
        print('This model does not properly compute!')

        
ar2_model = model_ARIMA(ts_series_ar2, order = (2, 0, 0))

                           Results: ARMA
Model:              ARMA             BIC:                 330.4571  
Dependent Variable: y                Log-Likelihood:      -158.05   
Date:               2021-08-31 20:10 Scale:               1.0000    
No. Observations:   120              Method:              mle       
Df Model:           2                Sample:              01-31-2005
Df Residuals:       118                                   12-31-2014
Converged:          1.0000           S.D. of innovations: 0.893     
No. Iterations:     8.0000           HQIC:                325.491   
AIC:                322.0946                                        
-----------------------------------------------------------------------
             Coef.     Std.Err.      t       P>|t|     [0.025    0.975]
-----------------------------------------------------------------------
ar.L1.y      0.6856      0.0881    7.7855    0.0000    0.5130    0.8581
ar.L2.y      0.2841      0.0891    3.1869    0.001

Note the following about the AR model:

- The estimated AR coefficients have values fairly close to the values used to generate the data. Further, true values are within the standard errors and confidence intervals of the estimated coefficients. Notice negative sign of the coefficient values. 
- The p-values are small and standard error ranges do not include 0 so the coefficient values are significant.


Now let's look at a moving average $\text{MA}(q)$ model. The code in the cell below computes an $\text{MA}(1)$ model with a coefficient $\beta_1 = 0.75$.

In [39]:
ts_series_ma1 = ARMA_process(ma_coef = [.75])
plot_ts_resid(ts_series_ma1)

Is the time series stationary? True
Is the time series invertible? True


The time series of the $\text{MA}(1)$ process looks fairly random, with no trend. 

Next, let's plot the ACF and PACF of the $\text{MA}(1)$ process. 

In [40]:
plot_acf_pacf(ts_series_ma1)

The ACF exhibits only one non-zero lag, which is expected for an $\text{MA}(1)$ process. There are also some significant non-zero lags in the PACF, which is a result of random noise.  

Let's try to estimate the coefficients of the MA time series. The code in the cell below fits and $\text{MA}(1)$ model to the time series. The model is specified as `(0, 0, q)`.

In [41]:
ma1_model = model_ARIMA(ts_series_ma1, order = (0, 0, 1))

                           Results: ARMA
Model:              ARMA             BIC:                 329.7539  
Dependent Variable: y                Log-Likelihood:      -160.09   
Date:               2021-08-31 20:10 Scale:               1.0000    
No. Observations:   120              Method:              mle       
Df Model:           1                Sample:              01-31-2005
Df Residuals:       119                                   12-31-2014
Converged:          1.0000           S.D. of innovations: 0.908     
No. Iterations:     8.0000           HQIC:                326.443   
AIC:                324.1789                                        
-----------------------------------------------------------------------
            Coef.     Std.Err.       t       P>|t|     [0.025    0.975]
-----------------------------------------------------------------------
ma.L1.y     0.9690      0.0258    37.4926    0.0000    0.9183    1.0196
--------------------------------------------------

Note the following about the AR model:

- The estimated MA coefficient has a value fairly close to the value used to generate the data. Further, true value is within the standard error and confidence intervals of the estimated coefficient. 
- The p-values are small and standard error ranges do not include 0 so the coefficient values are significant.


The code in the cell below simulates and plots an $\text{ARMA}(1, 1)$ model.

In [42]:
ts_series_arma11 = ARMA_process(ar_coef = [.6], ma_coef = [.75])
plot_ts_resid(ts_series_arma11)

Is the time series stationary? True
Is the time series invertible? True


As expected, the $\text{ARMA}(1, 1)$ series has properties of both an $\text{AR}(1)$ and $\text{MA}(1)$ series. 

And let's plot its ACF and PACF:

In [43]:
plot_acf_pacf(ts_series_arma11, lags = 40)

And once again let's estimate the model, specified by `(p, 0, q)`.

In [44]:
arma11_model = model_ARIMA(ts_series_arma11, (1, 0, 1))

                           Results: ARMA
Model:              ARMA             BIC:                 356.3088  
Dependent Variable: y                Log-Likelihood:      -170.97   
Date:               2021-08-31 20:10 Scale:               1.0000    
No. Observations:   120              Method:              mle       
Df Model:           2                Sample:              01-31-2005
Df Residuals:       118                                   12-31-2014
Converged:          1.0000           S.D. of innovations: 0.994     
No. Iterations:     9.0000           HQIC:                351.342   
AIC:                347.9463                                        
-----------------------------------------------------------------------
            Coef.     Std.Err.       t       P>|t|     [0.025    0.975]
-----------------------------------------------------------------------
ar.L1.y     0.5198      0.0835     6.2236    0.0000    0.3561    0.6835
ma.L1.y     0.9099      0.0726    12.5386    0.000

### Summary

In this notebook we explored basic properties of time series. We learned how ARMA models can be used with **stationary** data, and how we can decompose the data into a trend, a seasonal and a residual component and fit an ARMA model to its **residual component**. Alternatively, we learned how ARMA models can be extened to ARIMA by adding the **difference** operator, and SARIMA by adding a seasonal component. ARIMA can be used for non-stationary data with a trend, and SARIMA can be used with non-stationary data with a trend and seasonality. There are further extensions of these models that make it possible to model time series data when the variance changes over time, namely the **autoregressive conditional heteroskedasticity** or **ARCH** and **generalized autoregressive conditional heteroskedasticity** or **GARCH** model, but these models are beyond the scope of this notebook.

# Assignment

So far we have been studying time series that we generated using a pre-defined stochastic process, but now let's apply the models we have been working with on some real-world data. We will work with a data set which shows the consumption of chocolate, beer and electricity in Australia from 1958 to 1991.

In [45]:
from math import sin, pi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as nr

import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error

import statsmodels.graphics.tsaplots as splt
import statsmodels.api as statsmodels
import statsmodels.formula.api as sm
import statsmodels.tsa.seasonal as sts
import statsmodels.tsa.arima_process as arima_process
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 5]

import warnings
warnings.filterwarnings('ignore')

def decomp_ts(ts, period, model = 'additive'):
    res = sts.seasonal_decompose(ts, model = model, period = period)
    return(pd.DataFrame({'ts': ts, 'trend': res.trend, 'seasonal': res.seasonal, 'resid': res.resid}, 
                        index = ts.index))

def plot_acf_pacf(x, lags = 40):
    x = x[x.notna()] # remove NAs
    fig, axes = plt.subplots(1, 2, figsize = (15, 5))
    fig = splt.plot_acf(x, lags = lags, ax = axes[0])
    fig = splt.plot_pacf(x, lags = lags, ax = axes[1]);
    return None

def plot_ts_resid(x):
    x = x[x.notna()] # remove NAs
    fig, axes = plt.subplots(1, 2, figsize = (15, 5))
    fig = sns.lineplot(x.index, x, ax = axes[0])
    fig = sns.distplot(x, ax = axes[1]);
    return None

In [46]:
CBE = pd.read_csv('../data/cbe.csv')
CBE.index = pd.date_range(start = '1-1-1958', end = '12-31-1990', freq = 'M')

CBE.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/cbe.csv'

We limit our example to looking at beer consumption.

In [ ]:
plot_ts_resid(CBE['beer'])

Notice that for each of these time series the amplitude of the seasonal variation grows with time. This is a common situation with real world data. Seeing this situation indicates that we should use a **multiplicative decomposition model**.  

The multiplicative model can be easily transformed to an additive model by taking the logarithm of the values.

In [ ]:
CBE['beer_log'] = np.log(CBE['beer'])
plot_ts_resid(CBE['beer_log'])

Notice the following properties about this time series.
- It has a significant trend.
- The time series have a noticeable seasonal component.
- The magnitude of the seasonal component increases with trend in the un-transformed time series. 
- The seasonal component of the log transformed series has a nearly constant magnitude, but decreases a bit with time. 

These results indicate that an STL decomposition is required. Further, a multiplicative (log transformed) STL model is preferred.


### Fitting linear regression

Before training any time series model, let's see how our old fried linear regression does. In cases where the data is relatively well behaved, we can train a model using linear regression, but we need to do some pre-processing to account for the time series nature of the data. This can be a manual and laborious process, but going through it can give us a sense of what trying to model time series "manually" looks like.

- Create a feature called `month_int`, which is equal to 1 when the month is January, 2 for February, and so on. Create another feature called `month_sqr` which is the square of `month_int`. <span style="color:red" float:right>[2 point]</span>
- One-hot-encode the `month_int` feature (creating one binary feature for each month), and normalize `month_int` and `month_sqr`. <span style="color:red" float:right>[2 point]</span>
- Create a feature called `beer_log_lag_1` which is the first lag of `beer_log` (as in the last known price of beer, when you look at the previous row). HINT: You can get lagged features using the `shift` method. <span style="color:red" float:right>[2 point]</span>

In [ ]:
## your code goes here

With the feature engineering steps we took, we should be able to train a linear regression model now. With `month_int` and `month_sqr` the model should be able to find a trend over the course of the year, which is either linear or curvelinear with a single peak or trough. By one-hot-encoding `month_int` the model can also capture month to month effects. Finally, using a lagged feature, the model can anchor its beer price prediction on the last known price.

- Split the data into training and test sets, using the last 12 months of data for testing. <span style="color:red" float:right>[2 point]</span> 
- Train a linear regression model to predict beer price using onely the features we created earlier. <span style="color:red" float:right>[2 point]</span> 

In [ ]:
from sklearn.linear_model import LinearRegression

## your code goes here

- Plot a line plot of the original time series, and to the same plot add line plots to show the predictions on the training data and the test data. Use separate colors for each. <span style="color:red" float:right>[3 point]</span> 

In [ ]:
## your code goes here

- Compute the **root mean square error (RMSE)** of the model on the test data and plot the line plot and the histogram of the residual (beer price minus forecast) using the `plot_ts_resid` helper function. What conclusion would you draw about the model we fit? <span style="color:red" float:right>[2 point]</span> 

In [ ]:
from sklearn.metrics import mean_squared_error
## your code goes here

### Fitting a time series model

Let's now try the models we learned about in this lesson. By doing so, we can later compare the two approaches and appreciate the pros and cons.

- Use the `decomp_ts` helper function to decompose `beer_log`. Remove the NAs from the data, then use the `plot` method to plot a line plot of the components. <span style="color:red" float:right>[3 point]</span> 

In [ ]:
## your code goes here

- Compute and plot the ACF and PACF for the remainder (residual) series, up to 36 lags. <span style="color:red" float:right>[2 point]</span>

In [ ]:
## your code goes here

As you can imagine, with real data things can look very messy. The ACF and PACF can exhibit both AR and MA behavior and it's hard to know what degrees to choose. So we will use the `auto_arima` function to help us: It  iterates over a grid of $(p, d, q)$ and seasonal $(P, D, Q)$ values. For each combination the BIC is computed and compared to the best previous model. For each combination the BIC is computed and compared to the best previous model. The better model is the one with the lowest BIC: The **Bayesian information criteria (BIC)** is a measure for assessing a model's fit:

$$
\begin{align}
\text{BIC} &= \ln(n)k - 2 \ln(\hat L)
\end{align}
$$

where $\hat L$ is the likelihood of the data given the fitted model parmaters, $k$ is the number of model parameters, and $n$ is the number of observations. Lower values for BIC means we have a better fit.

The code below implements `auto_arima`. As you can see, we provide it with the data, some maximum value for the hyper-pramaters $(p, d, q)$ and $(P, D, Q)$. It's very unusual to choose a number greater than 3. Run the next cell and examine the results. The function returns the best model, i.e. the model whose hyper-parameters gave the lowest BIC.

In [ ]:
from pmdarima import auto_arima
best_fit = auto_arima(CBE.loc[:validation_cut_off, 'beer_log'], 
                      max_p = 3, max_d = 1, max_q = 3, 
                      m = 12, max_P = 1, max_D = 1, max_Q = 1, 
                      information_criterion = 'bic', 
                      trace = True, error_action = 'ignore', suppress_warnings = True)

Let's take a look at the best model's summary:

In [ ]:
print(best_fit.summary())

Let's now visualize the forecast. With time series models we use the `predict_in_sample` to make predictions for the range of data that we used during training, and we use `predict` to make forecasts.

In [ ]:
start_idx = 1
train_idx = X_train.reset_index().index[start_idx:]
n_periods = len(X_valid)

sns.lineplot(CBE.index, CBE['beer_log'], alpha = 0.3)
sns.lineplot(X_train.index[train_idx], best_fit.predict_in_sample(start = start_idx, end = train_idx.max()))
sns.lineplot(X_valid.index, best_fit.predict(n_periods = n_periods))
plt.legend(['original', 'fit', 'forecast']);

Notice how the predictions are initially a bit off, but overall the forecasts look reasonable.

- Compute the RMSE and use `plot_ts_resid` to plot the line plot and the histogram of the residuals. How does the RMSE for this model compare the the linear regression model? <span style="color:red" float:right>[2 point]</span>

In [ ]:
## your code goes here

We hope the assignment convinced you that the decision as to which model is better is not always clear. Of course we can rely on a metric like RMSE, but we don't want that to be the sole determinant. The level of familiarity with this or that algorithm should also be important. For example, we spent a lot of time learning about linear models, so even if the linear model is a slight worse fit we may prefer it because they are efficient and we can focus on feature engineering to improve its performance. ARIMA models on the other hand have the advantage of taking care of a lot of the feature engineering, but they are more difficult to explainr and require more experience in order to tune well. These sorts of trade-offs are very common in data science.

# End of assignment